In [1]:
from datasloth import DataSloth
import pandas as pd
import seaborn as sns

In [2]:
# Main dataset to show datasloth capabilities
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
# Make sure your OpenAI API key is set in the OPENAI_API_KEY env variable, or provide it as an argument to DataSloth()
sloth = DataSloth()

In [4]:
# Example 1: we do not need to specify exact lables in our data. Here, 'men' is autonatically converted to 'male'.
sloth.query("Number of men which survived the titanic", show_query=True)

SELECT COUNT(*) AS survived_men
FROM titanic
WHERE sex = 'male' AND survived = 1


,survived_men
0,109


In [5]:
# Exmaple 2: loosely specified statistics
sloth.query("Average fare paid by an men who traveled alone", show_query=True)

SELECT AVG(fare) AS avg_fare
FROM titanic
WHERE sex = 'male' AND alone = 1


,avg_fare
0,16.713358


In [6]:
# Example 3: more complex stats
sloth.query("Percentage of male survivors", show_query=True)

SELECT (SUM(CASE WHEN survived = 1 AND sex = 'male' THEN 1.0 END) / COUNT(*)) * 100 AS percentage
FROM titanic


,percentage
0,12.233446


In [7]:
# Example 4: group aggregations
sloth.query("Calculate the percentage of survivors per sex", show_query=True)

SELECT sex, (SUM(CASE WHEN survived = 1 THEN 1.0 END) / COUNT(*)) * 100 AS percentage
FROM titanic
GROUP BY sex


,sex,percentage
0,female,74.203822
1,male,18.890815


In [8]:
# Introducing another dataframe into the namespace
classes = pd.DataFrame({
    'pclass': [1, 1, 1, 2, 2, 2, 3, 3, 3],
    'meal_type': ['breakfast', 'lunch', 'dinner'] * 3, 
    'n_courses': [10, 15, 20, 5, 6, 7, 1, 2, 3]
})
classes

,pclass,meal_type,n_courses
0,1,breakfast,10
1,1,lunch,15
2,1,dinner,20
3,2,breakfast,5
4,2,lunch,6
5,2,dinner,7
6,3,breakfast,1
7,3,lunch,2
8,3,dinner,3


In [12]:
# Example 5: automatically joining with other tables in the namescpace
sloth.query("Calculate the percentage of survivors of people who had more than 5 courses for breakfast. Do it per sex.", show_query=True)

SELECT sex, (SUM(CASE WHEN survived = '1' THEN 1.0 END) / COUNT(*)) * 100 AS percentage
FROM titanic
JOIN classes ON titanic.pclass = classes.pclass
WHERE meal_type = 'breakfast' AND n_courses > 5
GROUP BY sex


,sex,percentage
0,female,96.808511
1,male,36.885246


In [13]:
# Another table, with departure dates from each port
# Note that the table and column names do not explain what the information is about
table_por_dep = pd.DataFrame({'code': ['S', 'C', 'Q'], 'date': pd.to_datetime(['1912-04-10', '1912-04-10', '1912-04-11'])})
table_por_dep

,code,date
0,S,1912-04-10
1,C,1912-04-10
2,Q,1912-04-11


In [14]:
# Sloth is not able to make the connection correctly, as it does not know that departure dates are stored in that other table
sloth.query("Count female passengers who departed on 11th of April", show_query=True)

SELECT COUNT(*) AS female_passengers
FROM titanic
WHERE sex = 'female'
AND date(embarked) = date('1912-04-11')


,female_passengers
0,0


In [15]:
# To help, we add the table description
# Note the use of a COLUMNS_SUMMARY placeholder to still keep the default description in.
table_por_dep.sloth.description = \
"Departure date table, to be joined to the main Titanic table on the 'embarked' code. \n{COLUMNS_SUMMARY}"

sloth.query("Count female passengers who departed from their port on 11th of April", show_query=True)

SELECT COUNT(*) AS female_passengers
FROM titanic
INNER JOIN table_por_dep ON titanic.embarked = table_por_dep.code
WHERE date(table_por_dep.date) = date('1912-04-11')
AND titanic.sex = 'female'


,female_passengers
0,36
